In [ ]:
import pandas as pd

In [ ]:
df_clients = pd.read_excel('case.xlsx', sheet_name='clients')
df_stores = pd.read_excel('case.xlsx', sheet_name='stores')
df_products = pd.read_excel('case.xlsx', sheet_name='products')
df_sales = pd.read_excel('case.xlsx', sheet_name='sales')
df_payments = pd.read_excel('case.xlsx', sheet_name='payments')

# First Analysis

In [ ]:
df_clients.sample(5)

In [ ]:
df_clients.head(5)

In [ ]:
df_clients.tail(5)

In [ ]:
df_clients.isnull().sum()

In [ ]:
df_clients[df_clients.isnull().T.any()]

In [ ]:
df_clients.sex.unique()

In [ ]:
df_products

In [ ]:
df_products.boxplot(column=['value'])

In [ ]:
df_products[df_products.value > 300000]

In [ ]:
df_products[df_products.value < 300000].boxplot(column=['value'])

In [ ]:
# discovering if that problematic product was sold
df_sales[df_sales.id_product==10].count()

In [ ]:
df_sales.isnull().sum()

In [ ]:
df_sales.describe()

In [ ]:
df_payments.isnull().sum()

In [ ]:
df_payments.describe()

# Data Cleaning

## Null Data

In [ ]:
df_clients[df_clients.name.isnull()]

In [ ]:
df_clients[df_clients.birth_date.isnull()]

In [ ]:
df_clients[df_clients.sex.isnull()]

In [ ]:
df_clients.loc[df_clients.name.isnull(), 'name'] = 'No Name'
df_clients.loc[df_clients.sex.isnull(), 'sex'] = 'O'
df_clients.loc[df_clients.birth_date.isnull(), 'birth_date'] = '1/1/2020'

In [ ]:
df_clients.loc[[269,287], :]

In [ ]:
# Now we have no null datag
df_clients.isnull().sum()

## Outliers

In [ ]:
df_products.loc[9,'value'] = df_products.value[9] / 10000
df_products.boxplot(column=['value'])

## Consistency

In [ ]:
df_sales[~df_sales.id_cliente.isin(df_clients.id)]

In [ ]:
df_sales[~df_sales.id_store.isin(df_stores.id)]

In [ ]:
df_sales[~df_sales.id_product.isin(df_products.id)]

In [ ]:
df_payments[~df_payments.id_sale.isin(df_sales.id)]

In [ ]:
df_sales[~df_sales.id.isin(df_payments.id_sale)]

## Duplicated Data

In [ ]:
df_clients[df_clients.name.duplicated()]

In [ ]:
df_clients.drop('id', axis=1).duplicated().sum()

In [ ]:
df_stores.city.duplicated().sum()

In [ ]:
df_products['product'].duplicated().sum()

In [ ]:
df_sales[df_sales.drop('id', axis=1).duplicated()]

In [ ]:
df_sales[(df_sales.id_cliente==559) & (df_sales.id_store==2) & (df_sales.id_product==5)]

In [ ]:
df_payments.drop('id', axis=1).duplicated().sum()

## Data Formatting

In [ ]:
df_clients.birth_date = pd.to_datetime(df_clients.birth_date, format='%m/%d/%Y')
df_clients.sample()

In [ ]:
df_payments.sample()

In [ ]:
df_products.sample()

In [ ]:
df_stores.sample()

## Indexes

In [ ]:
df_clients = df_clients.set_index('id')
df_stores = df_stores.set_index('id')
df_payments = df_payments.set_index('id')
df_products = df_products.set_index('id')
df_sales = df_sales.set_index('id')

# Data Fusion

In [ ]:
df = df_sales.join(df_clients.add_prefix('client_'), on='id_cliente')
df = df.join(df_stores.add_prefix('store_'), on='id_store')
df = df.join(df_products.add_prefix('product_'), on='id_product')
df.sample()

In [ ]:
df = df.join(df_payments.set_index('id_sale'))
df.sample()

In [ ]:
df.isnull().sum()

# Feature Engineering

In [ ]:
df['paid'] = 1
df.loc[df.dt_payment.isnull(), 'paid'] = 0
df

In [ ]:
df['days_to_pay'] = (df.dt_payment - df.dt_sale).dt.days
df

In [ ]:
import numpy as np
df['client_age'] = np.floor((pd.to_datetime('today') - df.client_birth_date)/np.timedelta64(1, 'Y'))
df

# Analysis

In [ ]:
graph_data = df.groupby('store_city').count().product_value.sort_values(ascending=False)

In [ ]:
import matplotlib.pyplot as plt
plt.figure(figsize=(15,5))
plt.bar(graph_data.index, graph_data.values)
plt.title('vendas por loja')

## Most sold products

In [ ]:
graph_product_sold = df.groupby('product_product').count().product_value.sort_values(ascending=False)
graph_product_sold

In [ ]:
plt.figure(figsize=(15,5))
plt.bar(graph_product_sold.index, graph_product_sold.values)
plt.title('Sales by Product')

## Store Analysis

In [ ]:
# These are the same, but I prefer the sort using the by, it seems clearer, but the first one leaves us a series, and the second one a dataframe...
df[['store_city','product_value']].groupby('store_city').sum().product_value.sort_values(ascending=False)
df[['store_city','product_value']].groupby('store_city').sum().sort_values(ascending=False, by="product_value")

In [ ]:
graph_data = df[['store_city','product_value']].groupby('store_city').sum().product_value.sort_values(ascending=False)
plt.figure(figsize=(15,5))
plt.title('Total amount of potential cash income by store')
plt.bar(graph_data.index, graph_data.values)

In [ ]:
max_value = graph_data.max()
others_mean = graph_data[graph_data!=graph_data.max()].mean()
times_greater_than_mean = graph_data.max()/others_mean
print('The max value is %i, and it is %.2f times greater than the mean of the other values, which is %i' % (max_value, times_greater_than_mean, others_mean))

In [ ]:
graph_data = df[['product_product', 'product_value']].groupby('product_product').sum().product_value.sort_values(ascending=False)
plt.figure(figsize=(15,5))
plt.title('Total amount of potential cash income by product')
plt.bar(graph_data.index, graph_data.values)

# Subplots

In [ ]:
plt.figure(figsize=(10,7))

plt.subplot(2,2,1)
sales_by_store_data = df.groupby('store_city').count().product_value.sort_values(ascending=False)
plt.bar(sales_by_store_data.index, sales_by_store_data.values)
plt.xticks(rotation=90)
plt.title('Sales By Store')

plt.subplot(2,2,2)
product_amount_sales = df.groupby('product_product').count().product_value.sort_values(ascending=False)
plt.bar(product_amount_sales.index, product_amount_sales.values)
plt.xticks(rotation=90)
plt.title('Quantity of Product Sales')

plt.subplot(2,2,3)
income_by_store_data = df.groupby('store_city').sum().product_value.sort_values(ascending=False)
plt.bar(income_by_store_data.index, income_by_store_data.values)
plt.xticks(rotation=90)
plt.title('Income by Store')

plt.subplot(2,2,4)
income_by_product_data = df.groupby('product_product').sum().product_value.sort_values(ascending=False)
plt.bar(income_by_product_data.index, income_by_product_data.values)
plt.xticks(rotation=90)
plt.title('Income by Product')

plt.tight_layout()


## Sales by Client

In [ ]:
sales_by_client_data = df[['client_name','product_value']].groupby('client_name').sum().product_value.sort_values(ascending=False)
print(sales_by_client_data)

In [ ]:
plt.figure(figsize=(15,5))
plt.plot(sales_by_client_data.index, sales_by_client_data.values)
frame = plt.gca().axes.get_xaxis().set_visible(False)
plt.title('Sales by Client')

In [ ]:
from matplotlib.ticker import PercentFormatter

figure, axis = plt.subplots(figsize=(15,5))
axis.plot(sales_by_client_data.index, sales_by_client_data.values, color='C0')
second_axis = axis.twinx()
second_axis.plot(sales_by_client_data.index, sales_by_client_data.values.cumsum()/sales_by_client_data.values.sum()*100, color='C1')
second_axis.yaxis.set_major_formatter(PercentFormatter())
second_axis.axes.get_xaxis().set_visible(False)
axis.axes.get_xaxis().set_visible(False)
plt.title('Sales By Client')

In [ ]:
sales_by_client_cumulative_total = sales_by_client_data.cumsum() / sales_by_client_data.sum()
print(sales_by_client_cumulative_total)

In [ ]:
sales_by_client_cumulative_total[sales_by_client_cumulative_total < 0.60].count() / sales_by_client_cumulative_total.count()
## 31 percent of our clients are responsible for 60 percent of our sales

## Pareto's Analysis

In [ ]:
income_by_product_data = df[['product_product','product_value']].groupby('product_product').sum().product_value.sort_values(ascending=False)
figure, axis = plt.subplots(figsize=(15,5))

axis.bar(income_by_product_data.index, income_by_product_data.values, color='C0')
second_axis = axis.twinx()
second_axis.plot(income_by_product_data.index, income_by_product_data.values.cumsum() / income_by_product_data.values.sum() * 100, color='C1', marker='D')
second_axis.yaxis.set_major_formatter(PercentFormatter())
plt.ylim(0,110)
plt.title("Pareto's Graph - Income by Product")

## Sales by product and store

In [ ]:
product_store_income_data = pd.DataFrame(columns=('store','product','income'))
for city in df_stores.city:
    for product in df_products['product']:
        new_data = {
            'store': city,
            'product': product ,
            'income': df.product_value[ (df.store_city == city) & (df.product_product == product) ].sum()
        }
        product_store_income_data = product_store_income_data.append(new_data, ignore_index=True)
print(product_store_income_data)

import seaborn as sns

data = product_store_income_data.pivot_table(index='store', columns='product', values='income', aggfunc='sum')
sns.heatmap(data)

In [ ]:
product_store_income_data = product_store_income_data.sort_values(by='income', ascending=False)
product_store_income_data['store-product'] = product_store_income_data['store'] + '-' + product_store_income_data['product'] 
filter_product_income = product_store_income_data.income > 0.01 * product_store_income_data.income.max()
plt.figure(figsize=(15,5))
plt.xticks(rotation=90)
plt.title('Income by Product')
plt.bar(product_store_income_data['store-product'][filter_product_income], product_store_income_data.income[filter_product_income])
plt.show()


## Time Analysis

In [ ]:
df.days_to_pay.mean()

In [ ]:
df.boxplot('days_to_pay')
plt.title('Boxplot for time between sale and payment')

In [ ]:
sns.histplot(data=df.days_to_pay, kde=True)
plt.title('Histogram for time between sale and payment')

In [ ]:
df.days_to_pay.describe()

## Payment time - Product and Store

In [ ]:
df.groupby('store_city').mean().days_to_pay

In [ ]:
plt.figure(figsize=(7,4))
plt.title('Boxplot - payment date by city')
df[['store_city','days_to_pay']].groupby('store_city').boxplot('days_to_pay')
plt.xticks(rotation=90)
plt.show()

In [ ]:
df.groupby('product_product').days_to_pay.mean()

In [ ]:
plt.figure(figsize=(7,4))
df[['product_product','days_to_pay']].groupby('product_product').boxplot('days_to_pay')
plt.title('Boxplot for time to pay between products ')
plt.xticks(rotation=90)
plt.show()